# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [ ]:
import json
import os
import logging
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

level = logging.WARNING

logger = logging.getLogger(__name__)
logging.basicConfig(level=level)
logger.setLevel(level)
logging.getLogger("llama_index").setLevel(level)
logging.getLogger("asyncio").setLevel(level)

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY")); openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [ ]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import (
    ExperimentalOpenAIAgent
)
from llama_index.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True
)

In [ ]:
#print(agent.chat("Tell me a random name"))

In [ ]:
#print(agent.chat("Now tell me that name backwards."))

In [ ]:
#response = exp_agent.chat("What is (121 * 3) + 42?")

### Chat

In [ ]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

In [ ]:
# inspect sources
print(response.sources)

### Async Chat

In [ ]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [6]:
from llama_index.agent.experimental_openai_agent import logger as _logger
assert logging.getLogger("llama_index.agent.experimental_openai_agent") is _logger

In [7]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a poem about a group of mice."
)

response_gen = response.response_gen

for ix, token in enumerate(response_gen):
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
In a world of whiskers and tiny paws,
There lived a group of mice, without a cause.
With fur so soft and eyes so bright,
They scurried and played, day and night.

Two hundred and forty-two, their number strong,
They danced and frolicked, a joyful throng.
Through fields of wheat and meadows green,
They explored the world, a sight unseen.

With nimble feet and curious minds,
They ventured far, leaving no trace behind.
In search of cheese and crumbs to eat,
They braved the darkness, their hearts replete.

Each mouse unique, with its own tale,
From timid to bold, they would prevail.
They built their nests in hidden nooks,
Safe from predators, with secret looks.

Their tiny voices, a chorus of glee,
Echoed through the night, wild and free.
They sang of friendship and unity,
A testament to their community.

Oh, the tales they wove, of daring feats,
Of narrow escapes and stolen treats.
With

In [9]:
response = exp_agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a poem about a group of mice."
)

response_gen = response.response_gen

for ix, token in enumerate(response_gen):
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
In a world of whiskers and tiny paws,
A group of mice scurried without pause.
With hearts so brave and spirits so free,
They embarked on adventures, a sight to see.

Two hundred and forty-two, their number strong,
They journeyed together, a lively throng.
Through fields and forests, they would roam,
Exploring the world, finding a home.

In meadows of green and flowers so bright,
They danced and played, day and night.
Their tiny footsteps left trails in the dew,
As they chased dreams that were bold and true.

With curiosity as their guiding light,
They discovered wonders, hidden from sight.
From secret tunnels to hidden nooks,
They explored every corner, like curious books.

Each mouse unique, with its own tale,
From timid whispers to adventures on a grand scale.
They shared laughter and tears, joys and fears,
Creating memories that would last for years.

Two hundred and forty-two, a 

### Async Streaming Chat

In [ ]:
response = await exp_agent.astream_chat(
    "What is 121 * 2 + 8? Once you have the answer, use that number to write a limerick about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)